# Sentiment Analysis for FakeNews

In [74]:
#Importing Pyforest as this is an automated library which has all the primary libraries
import pyforest

In [3]:
data = pd.read_csv("C:\\Users\\amith\\OneDrive\\Desktop\\Datasets\\FakeNews_train.csv")

<IPython.core.display.Javascript object>

In [4]:
data.head(5)

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [5]:
#Checking for null values
data.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [7]:
#Dropping the missing values as we cannot replace the missing values
data.dropna(inplace=True)

In [8]:
data.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [9]:
data.shape

(18285, 5)

In [12]:
#Dividing the dataset into dependent and independent features
#y is our dependent feature
y = data.iloc[:,-1]
y

0        1
1        0
2        1
3        1
4        1
        ..
20795    0
20796    0
20797    0
20798    1
20799    1
Name: label, Length: 18285, dtype: int64

In [30]:
#x is our dependent feature
x = data.iloc[:,1:-1]
x

,title,author,text
0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...
20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


IMPORTING ALL THE REQUIRED LIBRARIES

In [15]:
import tensorflow as tf

In [16]:
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [17]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [18]:
from tensorflow.keras.models import Sequential 

In [19]:
##Defining a vocabulary size, to specify all the unique words in our dictionary
##Here I am considering 1000 to make the OHE representation small, If higher the size, higher the representation
vocabulary_size = 1000

One Hot Encoding

In [49]:
#Here I am taking only title column to convert using OHE to predict the news
cor = x.copy()
cor.reset_index(inplace=True)

In [50]:
cor

,index,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
...,...,...,...,...
18280,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...
18281,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...
18282,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...
18283,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal..."


In [31]:
import re
import nltk

In [33]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\amith\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [34]:
#To use stemming importing PorterStemmer 
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [51]:
doc=[]
for i in range(0, len(cor)):
    new = re.sub('[^a-zA-Z]',' ', cor['title'][i])
    new = new.lower()
    new = new.split()
    
    new = [ps.stem(word) for word in new if not word in stopwords.words('english')]
    new = ' '.join(new)
    doc.append(new)

In [52]:
doc

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [53]:
#Performing One Hot Encoding
ohe = [one_hot(word,vocabulary_size) for word in doc] 

In [54]:
ohe

[[905, 212, 127, 437, 892, 504, 563, 676, 816, 782],
 [642, 682, 514, 597, 589, 694, 556],
 [633, 222, 160, 183],
 [738, 705, 251, 99, 348, 509],
 [418, 589, 963, 879, 869, 977, 589, 390, 336, 299],
 [848, 663, 841, 558, 160, 93, 967, 208, 837, 974, 341, 910, 908, 218, 556],
 [72, 388, 5, 953, 265, 528, 854, 38, 752, 399, 570],
 [310, 737, 629, 159, 908, 684, 93, 401, 752, 399, 570],
 [545, 227, 276, 460, 64, 840, 1, 400, 93, 899],
 [582, 371, 53, 745, 476, 98, 500, 241],
 [118, 406, 22, 518, 429, 835, 772, 796, 642, 968, 727],
 [99, 251, 892, 840, 93, 908],
 [480, 189, 868, 773, 91, 790, 53, 434, 888],
 [804, 963, 156, 398, 696, 350, 955, 752, 399, 570],
 [420, 600, 180, 771, 508, 752, 399, 570],
 [199, 946, 604, 75, 502, 279, 138, 189, 290, 691],
 [130, 224, 682],
 [404, 879, 549, 271, 93, 634, 150, 556],
 [959, 784, 514, 484, 917, 366, 90, 155, 977],
 [205, 930, 93, 383, 116, 556],
 [853, 359, 281, 53, 350, 90, 919, 627, 345, 752, 399, 570],
 [943, 816, 671, 386, 992, 58, 437],
 [90

# Word2Vec

In [55]:
#now implementing Padding to make all the sentece in doc in same length
sentece_length = 25
embedded_doc = pad_sequences(ohe,padding='post',maxlen=sentece_length)
embedded_doc

array([[905, 212, 127, ...,   0,   0,   0],
       [642, 682, 514, ...,   0,   0,   0],
       [633, 222, 160, ...,   0,   0,   0],
       ...,
       [188, 532, 870, ...,   0,   0,   0],
       [401, 908, 175, ...,   0,   0,   0],
       [142, 627, 204, ...,   0,   0,   0]])

# Model

In [57]:
embedded_vec_feature = 50
mod = Sequential()
mod.add(Embedding(vocabulary_size,embedded_vec_feature,input_length=sentece_length))
mod.add(LSTM(500))
mod.add(Dense(1,activation='sigmoid'))
mod.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
mod.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 25, 50)            50000     
                                                                 
 lstm_1 (LSTM)               (None, 500)               1102000   
                                                                 
 dense_1 (Dense)             (None, 1)                 501       
                                                                 
Total params: 1,152,501
Trainable params: 1,152,501
Non-trainable params: 0
_________________________________________________________________


In [58]:
final_x = np.array(embedded_doc)
final_y = np.array(y)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [59]:
final_x.shape

(18285, 25)

In [61]:
final_y.shape

(18285,)

In [62]:
#Splitting
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(final_x,final_y,test_size=0.25)

Training Model

In [63]:
mod.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=25,batch_size=75)

Epoch 1/25
183/183 [==============================] - 62s 301ms/step - loss: 0.3331 - accuracy: 0.8443 - val_loss: 0.2168 - val_accuracy: 0.9042
Epoch 2/25
183/183 [==============================] - 53s 290ms/step - loss: 0.2111 - accuracy: 0.9161 - val_loss: 0.2182 - val_accuracy: 0.9068
Epoch 3/25
183/183 [==============================] - 55s 301ms/step - loss: 0.1901 - accuracy: 0.9226 - val_loss: 0.2151 - val_accuracy: 0.9081
Epoch 4/25
183/183 [==============================] - 52s 286ms/step - loss: 0.1786 - accuracy: 0.9284 - val_loss: 0.2139 - val_accuracy: 0.9086
Epoch 5/25
183/183 [==============================] - 52s 286ms/step - loss: 0.1641 - accuracy: 0.9342 - val_loss: 0.2468 - val_accuracy: 0.9114
Epoch 6/25
183/183 [==============================] - 52s 286ms/step - loss: 0.1577 - accuracy: 0.9348 - val_loss: 0.2229 - val_accuracy: 0.9094
Epoch 7/25
183/183 [==============================] - 52s 286ms/step - loss: 0.1503 - accuracy: 0.9401 - val_loss: 0.2146 - val_ac

# Metrics

In [64]:
pred = mod.predict(X_test)
pred

143/143 [==============================] - 16s 52ms/step


array([[9.9787176e-01],
       [5.3312026e-05],
       [1.5659721e-01],
       ...,
       [8.8651641e-06],
       [9.9791145e-01],
       [9.9779361e-01]], dtype=float32)

In [66]:
#Setting the threshold as 0.7 so that we can get as accurate news as possible
pred = np.where(pred>0.7,1,0)
pred

<IPython.core.display.Javascript object>

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [68]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
cm = confusion_matrix(y_test,pred)
cm

array([[2387,  218],
       [ 223, 1744]], dtype=int64)

In [73]:
score = accuracy_score(y_test,pred)
score = score*100
score

90.35433070866141

In [71]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

           0       0.91      0.92      0.92      2605
           1       0.89      0.89      0.89      1967

    accuracy                           0.90      4572
   macro avg       0.90      0.90      0.90      4572
weighted avg       0.90      0.90      0.90      4572

